In [1]:
import uniplot as up
import pandas as pd
import numpy as np
import uvarprob as uvpr
import matplotlib.pyplot as plt
import solvutil as sut

In [2]:
test_file = "tst1.csv"

In [3]:
# Maximal number of steps
max_steps = 1e6
# Tolerance
epsilon = 1e-2
# If True - global Lipschitz constant is used
global_lipschitz_interval = True
# If True - the record value is taken from the test database
known_record = False
# How many points to skip in vizualization - regular step
skip = 1000
# The size of the legend in plots
legend_size = 2

In [4]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col = 'name', comment = '#')
    return data

In [5]:
df = read_problems(test_file)
df

,objective,a,b,min_f,min_x
name,,,,,
benchmark,x^3-10,-4,4,0,0


In [6]:
points_db = {}
psl_lipint_points_list = []
psl_lip_points_list = []
psqe_lipint_points_list =  []
psqe_lip_points_list = []

In [7]:
def log_point(x, points_list):
    points_list.append(x)

In [10]:
for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list' : [], 'psl_lip_points_list' : [], 'psqe_lipint_points_list' : [], 'psqe_lip_points_list' : []} 
#     print(points_db)

    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x, lambda x : log_point(x, points_db[test.Index]['psl_lipint_points_list']))
    # print(sut.psl(prob, sym = False))
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x, lambda x : log_point(x, points_db[test.Index]['psl_lip_points_list']))
    # print(sut.psl(prob, sym = True))
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x, lambda x : log_point(x, points_db[test.Index]['psqe_lipint_points_list']))
    print(sut.psqe(prob, sym = False))
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x, lambda x : log_point(x, points_db[test.Index]['psqe_lip_points_list']))
    # print(sut.psqe(prob, sym = True))


benchmark


KeyboardInterrupt: 

In [1]:
def vis_trials(points_list, num_series, base_level):
    poix = np.array(list(set(points_list[::skip])))
    poiy = np.full(len(poix), base_level + num_series * -0.5)
    up.plot_points(poix, poiy, num_series)

In [2]:
for test in df.itertuples():
    print(test)
    prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x)
    up.plot_problem(prob, {'function' : 'b-'}, legend = legend_size)
    vis_trials(points_db[test.Index]['psqe_lipint_points_list'], 0, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['psqe_lip_points_list'], 1, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['psl_lipint_points_list'], 2, plt.ylim()[0] + legend_size)
    vis_trials(points_db[test.Index]['psl_lip_points_list'], 3, plt.ylim()[0] + legend_size)
#     print("ylim = ", plt.ylim()[0])
    plt.show()


NameError: name 'df' is not defined